# Aula 1 - Reinforcement Learning

## Tutorial: Uma introdução ao aprendizado por reforço usando o táxi do Gymnasium 🚕

### Prof. Dr. Ahirton Lopes (profahirton.lopes@fiap.com.br)

Neste tutorial introdutório, aplicaremos aprendizagem por reforço (RL) para treinar um agente para resolver o [ambiente 'Táxi' do Gymnasium](https://gymnasium.farama.org/environments/toy_text/taxi/).

Abordaremos:

- Uma introdução básica ao RL;
- Configurando o Gymnasium & Taxi;
- Usando o algoritmo Q-learning para treinar nosso agente de táxi.

# Antes de começarmos, o que é 'Taxi'?

Táxi é um dos muitos ambientes disponíveis no Gymnasium. Esses ambientes são usados para desenvolver e avaliar algoritmos de aprendizagem por reforço.

O objetivo do Táxi é pegar os passageiros e deixá-los no destino com o menor número de movimentos.

Neste tutorial, vamos começar com um agente de táxi que executa ações aleatoriamente:

![agente aleatório](https://drive.google.com/uc?id=1l0XizDh9eGP3gVNCjJHrC0M3DeCWI8Fj)

…e aplicar com sucesso a aprendizagem por reforço para resolver o ambiente:

![agente treinado](https://drive.google.com/uc?id=1a-OeLhXi3W-kvQuhGRyJ1dOSw4vrIBxr)

# 💡 Uma introdução ao Aprendizado por Reforço

Pense em como você pode ensinar um novo truque a um cachorro como, por exemplo, mandá-lo sentar:

- Se ele executar o truque corretamente (sentar), você o recompensará com uma guloseima (feedback positivo) ✔️
- Se não assentar corretamente, não recebe tratamento (feedback negativo) ❌

Ao continuar a fazer coisas que levam a resultados positivos, o cão aprenderá a sentar-se ao ouvir o comando para receber a guloseima. O aprendizado por reforço é um subdomínio do aprendizado de máquina que envolve treinar um 'agente' (o cachorro) para aprender as sequências corretas de ações a serem executadas (sentado) em seu ambiente (em resposta ao comando 'sentar'), a fim de maximizar sua recompensa. (recebendo uma guloseima). Isso pode ser ilustrado mais formalmente como:

![sutton barto rl](https://www.gocoder.one/static/RL-diagram-b3654cd3d5cc0e07a61a214977038f01.png "Diagrama de aprendizado por reforço")

Fonte: [Sutton & Barto](http://incompleteideas.net/book/bookdraft2017nov5.pdf)

# 🏋️ Instalando o Gymnasium e Taxi

Usaremos o ambiente 'Taxi-v3' para este tutorial. Para instalar o gym (e numpy para depois), execute a célula abaixo:


In [11]:
!pip install gymnasium
!pip install numpy

Em seguida, importe o gym (e bibliotecas adicionais que serão úteis posteriormente):

In [12]:
import gymnasium as gym
import numpy as np
import random

# used to help with visualizing in Colab
from IPython.display import display, clear_output
from time import sleep

Gym contém uma grande biblioteca de diferentes ambientes. Vamos criar o ambiente Taxi-v3:

In [13]:
# creating Taxi environment with Gymnasium
env = gym.make('Taxi-v3', render_mode='ansi')

# 🎲 Agente aleatório

Começaremos implementando um agente que não aprende nada. Em vez disso, selecionará ações aleatoriamente. Ele servirá como nosso *baseline*.

O primeiro passo é dar ao nosso agente a observação inicial do estado. Um estado informa ao nosso agente como é o ambiente atual.

No Táxi, um estado define as posições atuais do táxi, do passageiro e dos locais de embarque e desembarque. Abaixo estão exemplos de três estados diferentes para táxi:

![estados de táxi](https://www.gocoder.one/static/taxi-states-0aad1b011cf3fe07b571712f2123335c.png "Diferentes estados de táxi")

Nota: Amarelo = táxi, Letra azul = local de retirada, Letra roxa = destino de entrega

Para obter o estado inicial:

In [14]:
# create a new instance of taxi, and get the initial state
state = env.reset()

print(f"Initial state: {state}")

Initial state: (442, {'prob': 1.0, 'action_mask': array([0, 1, 0, 1, 0, 0], dtype=int8)})


A seguir, executaremos um loop for para percorrer o jogo. Em cada iteração, nosso agente irá:

1. Fazer uma ação aleatória a partir do espaço de ação (0 - sul, 1 - norte, 2 - leste, 3 - oeste, 4 - recolha, 5 - desembarque)
2. Receber o novo estado

Aqui está nosso agente aleatório:

In [15]:
num_steps = 99
for s in range(num_steps+1):

    clear_output(wait=True)

    print(f"step: {s} out of {num_steps}")

    # sample a random action from the list of available actions
    action = env.action_space.sample()

    # perform this action on the environment
    env.step(action)

    # print the new state
    env.render()

    sleep(0.2)

# end this instance of the taxi environment
env.close()

step: 99 out of 99


Ao executar a célula acima, você verá seu agente fazendo movimentos aleatórios. Não é muito emocionante, mas espero que tenha ajudado você a se familiarizar com o kit de ferramentas Gymnasium.

A seguir, implementaremos o algoritmo Q-learning que permitirá ao nosso agente aprender com as recompensas.

# 📖 Agente Q-Learning

Q-learning é um algoritmo de aprendizagem por reforço que busca encontrar a melhor próxima ação possível dado seu estado atual, a fim de maximizar a recompensa que recebe (o 'Q' em Q-learning significa qualidade - ou seja, quão valiosa é uma ação) .

Vamos considerar o seguinte estado inicial:

![estado do táxi](https://www.gocoder.one/static/start-state-6a115a72f07cea072c28503d3abf9819.png "Um exemplo de estado do táxi")

Que ação (para cima, para baixo, para a esquerda, para a direita, para pegar ou largar) ele deve realizar para maximizar sua recompensa? (_Nota: azul = local de retirada e roxo = destino de entrega_)

Primeiro, vamos dar uma olhada em como nosso agente é “recompensado” por suas ações. **Lembre-se de que, no aprendizado por reforço, queremos que nosso agente execute ações que maximizem as possíveis recompensas que ele recebe de seu ambiente.**

## Sistema de recompensas "Táxi"

De acordo com a [documentação do táxi](https://gymnasium.farama.org/environments/toy_text/taxi/):

> _"…você recebe +20 pontos por uma entrega bem-sucedida e perde 1 ponto para cada intervalo de tempo necessário. Há também uma penalidade de 10 pontos para ações ilegais de coleta e entrega."_

Olhando para o nosso estado original, as ações possíveis que ele pode realizar e as recompensas correspondentes que receberá são mostradas abaixo:

![recompensas de táxi](https://www.gocoder.one/static/state-rewards-62ab43a53e07062b531b3199a8bab5b3.png "Recompensas de táxi")

Na imagem acima, o agente perde 1 ponto por timestep que realiza. Ele também perderá 10 pontos se usar a ação de retirada ou entrega aqui.

Queremos que nosso agente vá para o norte em direção ao local de coleta indicado por um R azul- **mas como ele saberá qual ação tomar se todos forem igualmente punitivos?**

## Exploração (Exploration)

Atualmente, nosso agente não tem como saber qual ação o levará mais próximo do R azul. É aqui que entra a tentativa e erro - faremos nosso agente realizar ações aleatórias e observar quais recompensas ele recebe (ou seja, nosso agente irá **explorar**).

Ao longo de muitas iterações, nosso agente terá observado que certas sequências de ações serão mais gratificantes que outras. Ao longo do caminho, nosso agente precisará acompanhar quais ações levaram a quais recompensas.

## Apresentando… tabelas Q

Uma tabela Q é simplesmente uma tabela de consulta que armazena valores que representam as recompensas futuras máximas esperadas que nosso agente pode esperar para uma determinada ação em um determinado estado (_conhecidos como valores Q_). Isso dirá ao nosso agente que, quando ele encontra um determinado estado, algumas ações têm maior probabilidade do que outras de levar a recompensas mais altas. Torna-se uma 'folha de dicas' informando ao nosso agente qual é a melhor ação a ser tomada.

A imagem abaixo ilustra como será a nossa 'tabela Q':

- Cada linha corresponde a um estado único no ambiente 'Táxi'
- Cada coluna corresponde a uma ação que nosso agente pode realizar
- Cada célula corresponde ao valor Q para esse par estado-ação - um valor Q mais alto significa uma recompensa máxima mais alta que nosso agente pode esperar obter se realizar essa ação naquele estado.

![Tabela Q](https://www.gocoder.one/static/q-table-9461cc903f50b78d757ea30aeb3eb8bc.png "Tabela Q")

Antes de começarmos a treinar nosso agente, precisaremos inicializar nossa tabela Q da seguinte forma:

In [16]:
state_size = env.observation_space.n  # total number of states (S)
action_size = env.action_space.n      # total number of actions (A)

# initialize a qtable with 0's for all Q-values
qtable = np.zeros((state_size, action_size))

print(f"Q table: {qtable}")

Q table: [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


À medida que nosso agente explora, ele atualizará a tabela Q com os valores Q que encontrar. Para calcular nossos valores Q, apresentaremos o algoritmo Q-learning.

# Algoritmo Q-Learning

O algoritmo Q-learning é fornecido abaixo. Não entraremos em detalhes, mas você pode ler mais sobre isso no [Capítulo 6 de Sutton & Barto (2018)](http://www.incompleteideas.net/book/RLbook2018trimmed.pdf).

![Algoritmo de aprendizagem Q](https://www.gocoder.one/static/q-learning-algorithm-84b84bb5dc16ba8097e31aff7ea42748.png "O algoritmo de aprendizagem Q")

O algoritmo Q-learning ajudará nosso agente a **atualizar o valor Q atual ($Q(S_t,A_t)$) com suas observações após realizar uma ação.** Ou seja, aumente Q se encontrar uma recompensa positiva ou diminua Q se encontrar uma recompensa negativa.

Observe que no Táxi, nosso agente não recebe uma recompensa positiva até que deixe um passageiro com sucesso (_+20 pontos_). Portanto, mesmo que nosso agente esteja indo na direção correta, haverá um atraso na recompensa positiva que deveria receber. O seguinte termo na equação Q-learning aborda isso:

![q máximo](https://www.gocoder.one/static/max-q-e593ddcec76cda87ed189c31d60837b6.png "Valor máximo de Q")

Este termo ajusta nosso valor Q atual para incluir uma parte das recompensas que ele poderá receber em algum momento no futuro (St+1). O termo 'a' refere-se a todas as ações possíveis disponíveis para esse estado. A equação também contém dois hiperparâmetros que podemos especificar:

1. Taxa de aprendizagem (α): quão facilmente o agente deve aceitar novas informações em vez de informações aprendidas anteriormente
2. Fator de desconto (γ): quanto o agente deve levar em consideração as recompensas que poderá receber no futuro versus sua recompensa imediata

Aqui está nossa implementação do algoritmo Q-learning:

In [17]:
# hyperparameters to tune
learning_rate = 0.9
discount_rate = 0.8

# dummy variables
reward = 10 # R_(t+1)
state = env.observation_space.sample()      # S_t
action = env.action_space.sample()          # A_t
new_state = env.observation_space.sample()  # S_(t+1)

# Qlearning algorithm: Q(s,a) := Q(s,a) + learning_rate * (reward + discount_rate * max Q(s',a') - Q(s,a))
qtable[state, action] += learning_rate * (reward + discount_rate * np.max(qtable[new_state,:]) - qtable[state,action])

print(f"Q-value for (state, action) pair ({state}, {action}): {qtable[state,action]}")

Q-value for (state, action) pair (459, 0): 9.0


## Comparação entre Exploration e Exploitation (Trade Off)

Podemos deixar nosso agente explorar para atualizar nossa tabela Q usando o algoritmo Q-learning. À medida que nosso agente aprende mais sobre o ambiente, podemos deixá-lo usar esse conhecimento para realizar ações mais otimizadas e convergir mais rapidamente - conhecido como **exploitation**.

Durante o exploitation, nosso agente examinará sua tabela Q e selecionará a ação com o valor Q mais alto (em vez de uma ação aleatória). Com o tempo, nosso agente precisará explorar menos e, em vez disso, começar "exploitar" o que sabe.

Aqui está nossa implementação de uma estratégia de exploration-exploitation:

In [18]:
# dummy variables
episode = random.randint(0,500)
qtable = np.random.randn(env.observation_space.sample(), env.action_space.sample())

# hyperparameters
epsilon = 1.0     # probability that our agent will explore
decay_rate = 0.01 # of epsilon

if random.uniform(0,1) < epsilon:
    # explore
    action = env.action_space.sample()
else:
    # exploit
    action = np.argmax(qtable[state,:])

# epsilon decreases exponentially --> our agent will explore less and less
epsilon = np.exp(-decay_rate*episode)

No exemplo acima, definimos algum valor `épsilon` entre 0 e 1. Se `épsilon` for 0,7, há 70% de chance de que nesta etapa nosso agente explore em vez de exploit. `epsilon` decai exponencialmente a cada passo, de modo que nosso agente explora cada vez menos ao longo do tempo.

# Reunindo tudo

Concluímos todos os blocos de construção necessários para nosso agente de aprendizagem por reforço. O processo de treinamento do nosso agente será semelhante a:

1. Inicializando nossa tabela Q com 0 para todos os valores Q
2. Deixe nosso agente jogar Taxi em um grande número de jogos
3. Atualizar continuamente a tabela Q usando o algoritmo Q-learning e uma estratégia de exploration-exploitation

Aqui está a implementação completa:

In [21]:
# class for colors on console
class bcolors:
    RED = '\u001b[31m'
    GREEN = '\u001b[32m'
    RESET = '\u001b[0m'

# create Taxi environment
env = gym.make('Taxi-v3', render_mode='ansi')

# initialize q-table
state_size = env.observation_space.n
action_size = env.action_space.n
qtable = np.zeros((state_size, action_size))

# hyperparameters
learning_rate = 0.9
discount_rate = 0.8
epsilon = 1.0
decay_rate= 0.005

# training variables
num_episodes = 5000
max_steps = 99 # per episode

print("AGENT IS TRAINING...")

for episode in range(num_episodes):

	# Reset the environment and get initial state
	state, info = env.reset(seed=42)
	step = 0
	done = False

	for step in range(max_steps):

		# Exploration-exploitation tradeoff
		if random.uniform(0,1) < epsilon:
			# Explore
			action = env.action_space.sample()
		else:
			# Exploit
			action = np.argmax(qtable[state,:])

		# Take an action and observe the reward
		next_state, reward, done, truncated, info = env.step(action)

		# Q-learning algorithm
		qtable[state, action] = qtable[state, action] + learning_rate * (reward + discount_rate * np.max(qtable[next_state, :]) - qtable[state, action])

		# Update to our new state
		state = next_state

		# if done, finish episode
		if done or truncated:
			break

	# Decrease epsilon
	epsilon = np.exp(-decay_rate*episode)

# Get ready to watch our trained agent
clear_output()
print(f"Our Q-table: {qtable}")
print(f"Training completed over {num_episodes} episodes")
input("Press Enter to see our trained taxi agent")
sleep(1)
clear_output()

episodes_to_preview = 3
for episode in range(episodes_to_preview):

	# Reset the environment
	state, info = env.reset(seed=42)
	step = 0
	done = False
	episode_rewards = 0

	for step in range(max_steps):
		# clear screen
		clear_output(wait=True)

		print(f"TRAINED AGENT")
		print(f"+++++EPISODE {episode+1}+++++")
		print(f"Step {step+1}")

		# Exploit
		action = np.argmax(qtable[state,:])

		# Take an action and observe the reward
		next_state, reward, done, truncated, info = env.step(action)

		# Accumulate our rewards
		episode_rewards += reward

		print(env.render())
		print("")
		if episode_rewards < 0:
			print(f"Score: {bcolors.RED}{episode_rewards}{bcolors.RESET}")
		else:
			print(f"Score: {bcolors.GREEN}{episode_rewards}{bcolors.RESET}")
		sleep(0.5)

		# Update to our new state
		state = next_state

		# if done, finish episode
		if done or truncated:
			break

# Close the Taxi environment
env.close()

TRAINED AGENT
+++++EPISODE 3+++++
Step 13
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


Score: 8


# 👏 O que vem a seguir?

Existem muitos outros ambientes disponíveis no Gymnasium para você experimentar (por exemplo, [Frozen Lake](https://gymnasium.farama.org/environments/toy_text/frozen_lake/)). Você também pode tentar otimizar a implementação acima para resolver o Táxi em menos passos.

Alguns outros recursos úteis incluem:
- [Série de palestras de aprendizagem por reforço DeepMind x UCL [2021]](https://www.youtube.com/watch?v=TCCjZe0y4Qc&ab_channel=GoogleDeepMind) (no Youtube)
- [Uma (longa) espiada na aprendizagem por reforço](https://lilianweng.github.io/lil-log/2018/02/19/a-long-peek-into-reinforcement-learning.html) por Lilian Weng
- [Um bom artigo sobre RL e suas aplicações no mundo real](https://www.altexsoft.com/blog/datascience/reinforcement-learning-explained-overview-comparisons-and-applications-in-business/)
- [Documentário completo do AlphaGo](https://www.youtube.com/watch?v=WXuK6gekU1Y) (no Youtube)
- [Aprendizagem por Reforço](http://www.incompleteideas.net/book/RLbook2018trimmed.pdf) por Sutton e Barto
- [Introdução prática ao aprendizado por reforço profundo](https://www.gocoder.one/blog/hands-on-introduction-to-deep-reinforcement-learning)

# O que resolvemos via Reinforcement Learning?

* Programação de elevador
* Passeio de bicicleta
* Direção de navio
* Controle de biorreator
* Controle de helicóptero de acrobacias
* Programação de partidas de aeroporto
* Regulamentação e preservação de ecossistemas
* Futebol Robocup
* Jogo de videogame (Atari, Starcraft...)
* Jogo de Go